In [2]:
import pandas as pd
import numpy as np

In [3]:
import sys

if 'google.colab' in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')
  path_prefix = '/content/drive/MyDrive/Great Learning/DBA/Walsh-MS-QM640CapstoneProject/'
  print("Running in Google Colab.")
else:
  path_prefix = './'
  print("Not running in Google Colab.")

print(f"Using path prefix: {path_prefix}")

Not running in Google Colab.
Using path prefix: ./


In [4]:
df = pd.read_excel(f"{path_prefix}scored/reviews_predicted_eng.xlsx")
print(df.shape)
df.head()

(53719, 20)


,reviewId,reviewContent,rating,reviewedAt,replyContent,repliedAt,store,app_name,at_year,at_month,country,at_calc,at_eta_date,reply_in,reply_eta,sla_status,responded_by,language_pred,department_pred,sentiment_pred
0,a2393404-7755-4564-add7-b3029159bb71,the application now for 4 days is not opening ...,1,2025-07-01 02:04:14,Sorry to hear you're having trouble with the a...,2025-07-01 12:49:59,Google Play Store,ADCB,2025,7,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,4.833056,0.298623,SLA Met,A.E,en,Application,Negative
1,f521326b-0ffb-4620-8995-b04eeee01c8f,normal,3,2025-06-30 22:17:54,NaN,NaT,Google Play Store,ADCB,2025,6,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,NaN,-1000.000000,Not Responded,NaN,en,General,Neutral
2,6262b57b-7c21-4f6b-9a6a-b4fa8d60b395,good,5,2025-06-30 21:42:32,"Hello, thank you for the encouraging star rati...",2025-07-01 12:49:03,Google Play Store,ADCB,2025,6,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,4.817500,0.299271,SLA Met,A.E,en,General,Positive
3,1780c47d-38ad-4563-92c3-c381b717171f,good thank you,5,2025-06-30 19:14:18,"Hello, thank you for the encouraging star rati...",2025-07-01 12:49:00,Google Play Store,ADCB,2025,6,NaN,2025-06-30 19:14:18,2025-07-01 19:14:18,5.578333,0.267569,SLA Met,A.E,en,General,Positive
4,5083ac69-2298-45ee-9b96-1912e10dfed6,Dear adcb bank i had a trouble on the Applicat...,5,2025-06-30 19:08:16,Sorry to hear you're having trouble with the a...,2025-07-01 12:49:52,Google Play Store,ADCB,2025,6,NaN,2025-06-30 19:08:16,2025-07-01 19:08:16,5.693333,0.262778,SLA Met,A.E,en,Application,Negative


In [5]:
# Rename columns for clarity in RAG pipeline context
df = df.rename(columns={
    'reviewId': 'review_id',
    'reviewContent': 'review_text',
    'rating': 'star_rating',
    'reviewedAt': 'review_date',
    'replyContent': 'reply_text',
    'repliedAt': 'reply_date',
    'store': 'app_store',
    'app_name': 'app_name',
    'at_year': 'year',
    'at_month': 'month',
    'country': 'country',
    'at_calc': 'calc_date',
    'at_eta_date': 'eta_date',
    'reply_in': 'reply_time_hrs',
    'reply_eta': 'reply_eta',
    'sla_status': 'sla_status',
    'responded_by': 'responded_by',
    'language_pred': 'language',
    'department_pred': 'department',
    'sentiment_pred': 'sentiment'
})

print("Columns renamed for RAG pipeline clarity.")
df.head()

Columns renamed for RAG pipeline clarity.


,review_id,review_text,star_rating,review_date,reply_text,reply_date,app_store,app_name,year,month,country,calc_date,eta_date,reply_time_hrs,reply_eta,sla_status,responded_by,language,department,sentiment
0,a2393404-7755-4564-add7-b3029159bb71,the application now for 4 days is not opening ...,1,2025-07-01 02:04:14,Sorry to hear you're having trouble with the a...,2025-07-01 12:49:59,Google Play Store,ADCB,2025,7,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,4.833056,0.298623,SLA Met,A.E,en,Application,Negative
1,f521326b-0ffb-4620-8995-b04eeee01c8f,normal,3,2025-06-30 22:17:54,NaN,NaT,Google Play Store,ADCB,2025,6,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,NaN,-1000.000000,Not Responded,NaN,en,General,Neutral
2,6262b57b-7c21-4f6b-9a6a-b4fa8d60b395,good,5,2025-06-30 21:42:32,"Hello, thank you for the encouraging star rati...",2025-07-01 12:49:03,Google Play Store,ADCB,2025,6,NaN,2025-07-01 08:00:00,2025-07-01 20:00:00,4.817500,0.299271,SLA Met,A.E,en,General,Positive
3,1780c47d-38ad-4563-92c3-c381b717171f,good thank you,5,2025-06-30 19:14:18,"Hello, thank you for the encouraging star rati...",2025-07-01 12:49:00,Google Play Store,ADCB,2025,6,NaN,2025-06-30 19:14:18,2025-07-01 19:14:18,5.578333,0.267569,SLA Met,A.E,en,General,Positive
4,5083ac69-2298-45ee-9b96-1912e10dfed6,Dear adcb bank i had a trouble on the Applicat...,5,2025-06-30 19:08:16,Sorry to hear you're having trouble with the a...,2025-07-01 12:49:52,Google Play Store,ADCB,2025,6,NaN,2025-06-30 19:08:16,2025-07-01 19:08:16,5.693333,0.262778,SLA Met,A.E,en,Application,Negative


In [6]:
df['sla_status'].unique()

array(['SLA Met', 'Not Responded', 'SLA Missed'], dtype=object)

In [5]:
%pip install sentence-transformers faiss-cpu transformers torch

In [7]:
# Install required libraries (uncomment if running for the first time)
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import torch
import openai
import os
import re
print('Libraries imported for RAG pipeline.')

Libraries imported for RAG pipeline.


In [10]:
# Generate embeddings for review_text using OpenAI text-embedding-3-large
from tqdm import tqdm

openai_embed_model = "text-embedding-3-large"
openai_client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Drop NA reviews for embedding
texts = df['review_text'].fillna('').tolist()

# Batch embedding for efficiency
def get_openai_embeddings(texts, batch_size=100):
    all_embeds = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = [str(t) if t is not None else "" for t in texts[i:i+batch_size]]
        # Remove empty strings (OpenAI API does not accept them)
        batch = [t for t in batch if t.strip()]
        if not batch:
            continue
        response = openai_client.embeddings.create(
            input=batch,
            model=openai_embed_model
        )
        batch_embeds = [d.embedding for d in response.data]
        all_embeds.extend(batch_embeds)
    return np.array(all_embeds)

In [ ]:
embeddings = get_openai_embeddings(texts)

# Save embeddings to disk for reuse
embeddings_path = f"{path_prefix}scored/review_embeddings_openai.npy"
np.save(embeddings_path, embeddings)
print(f"Embeddings saved to {embeddings_path}. To reload, use: np.load(embeddings_path)")

100%|██████████| 538/538 [19:40<00:00,  2.19s/it]


Embeddings saved to ./scored/review_embeddings_openai.npy. To reload, use: np.load(embeddings_path)


In [8]:
# Reload embeddings from disk
embeddings_path = f"{path_prefix}scored/review_embeddings_openai.npy"
embeddings = np.load(embeddings_path)
print(f"Embeddings reloaded from {embeddings_path}. Shape: {embeddings.shape}")

Embeddings reloaded from ./scored/review_embeddings_openai.npy. Shape: (53719, 3072)


In [13]:
# Build FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print(f"FAISS index built with {len(texts)} reviews using OpenAI text-embedding-3-large.")

faiss.write_index(index, f"{path_prefix}scored/review_faiss_index.bin")
print("FAISS index saved to disk.")

FAISS index built with 53719 reviews using OpenAI text-embedding-3-large.
FAISS index saved to disk.


In [14]:
## Load FAISS index from disk

index = faiss.read_index(f"{path_prefix}scored/review_faiss_index.bin")
print("FAISS index loaded from disk.")

FAISS index loaded from disk.


In [ ]:
from transformers import pipeline

# Load a small, fast model for intent classification (can use distilbert-base-uncased or similar)
intent_classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

c:\Users\user\anaconda3\envs\gen_ai\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fallin

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Device set to use cpu


In [66]:
client = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Hybrid RAG pipeline: intent detection, semantic & structured (pandas) search, LLM-assisted text-to-pandas

def detect_intent(prompt):
    """Very simple intent detection: if prompt contains keywords, treat as structured; else semantic."""
    structured_keywords = ["show", "which", "list", "count", "number of", "average", "sum ", "group by", "table", "sql", "pandas"]
    for kw in structured_keywords:
        if kw in prompt.lower():
            return "structured"
    return "semantic"

def detect_intent_model(prompt):
    # Predict intent using the classifier
    result = intent_classifier(prompt)[0]
    # Map model output to your intents (customize as needed)
    if result['label'] == 'POSITIVE':
        return "structured"
    else:
        return "semantic"
    
def resolve_intent(prompt):
    # Get intent from both methods
    intent_keyword = detect_intent(prompt)
    print("Intent detection using simple keywords:", intent_keyword)
    intent_model = detect_intent_model(prompt)
    print("Intent detection using pretrained classifier:", intent_model)

    # If both agree, use that intent
    if intent_keyword == intent_model:
        return intent_keyword
    # If model is confident (e.g., high score), prefer model
    # You can modify detect_intent_model to return (intent, score) for more control
    # For now, if model says 'structured', prefer it for tabular/aggregation queries
    if intent_model == "structured" and any(kw in prompt.lower() for kw in ["show", "list", "count", "group by", "table", "sql", "pandas"]):
        return "structured"
    # Otherwise, default to keyword-based intent
    return intent_keyword

# Column descriptions for the DataFrame
df_column_descriptions = {
    'review_id': 'Unique review identifier',
    'review_text': 'Text of the customer review',
    'star_rating': 'Star rating given by the user (numeric)',
    'review_date': 'Date of the review (datetime)',
    'reply_text': 'Text of the reply from support (if any)',
    'reply_date': 'Date of the reply (datetime, if any)',
    'app_store': 'App store source (Google Play or Apple)',
    'app_name': 'Name of the app',
    'year': 'Year of the review',
    'month': 'Month of the review',
    'country': 'Country of the reviewer',
    'calc_date': 'Calculated date field (if present)',
    'eta_date': 'Estimated time of arrival date (if present)',
    'reply_time_days': 'Reply time in days',
    'reply_eta': 'Reply ETA (if present)',
    'sla_status': 'SLA status for the reply',
    'responded_by': 'Name or ID of the responder',
    'language': 'Predicted language of the review',
    'department': 'Predicted department for the review',
    'sentiment': 'Predicted sentiment label',
}

# LLM-assisted text-to-pandas code generation
def generate_pandas_code(prompt, df_name="df"):
    # Get unique values for key columns
    sentiment_values = df['sentiment'].unique().tolist() if 'sentiment' in df.columns else []
    department_values = df['department'].unique().tolist() if 'department' in df.columns else []
    sla_status_values = df['sla_status'].unique().tolist() if 'sla_status' in df.columns else []
    language_values = df['language'].unique().tolist() if 'language' in df.columns else []
    app_name_values = df['app_name'].unique().tolist() if 'app_name' in df.columns else []
    app_store_values = df['app_store'].unique().tolist() if 'app_store' in df.columns else []

    column_info = "\n".join([f"{col}: {desc}" for col, desc in df_column_descriptions.items()])
    value_info = (
        f"sentiment possible values: {sentiment_values}\n"
        f"department possible values: {department_values}\n"
        f"sla_status possible values: {sla_status_values}\n"
        f"language possible values: {language_values}\n"
        f"app_name possible values: {app_name_values}\n"
        f"app_store possible values: {app_store_values}"
    )
    system_msg = (
        "You are a Python data analyst. Given a user request and a pandas DataFrame called 'df', "
        "write a single line of pandas code to answer the request. Only output the code, no explanation.\n"
        f"DataFrame columns and descriptions:\n{column_info}\n"
        f"{value_info}"
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": prompt}
        ],
        max_tokens=128,
        temperature=0.0
    )
    code = response.choices[0].message.content.strip()
    return code

def hybrid_rag(prompt, k=10):
    intent = resolve_intent(prompt)
    if intent == "semantic":
        # Use semantic RAG pipeline
        print("Using semantic RAG pipeline")
        query_emb = openai_client.embeddings.create(
            input=[prompt],
            model=openai_embed_model
        ).data[0].embedding
        query_emb = np.array(query_emb, dtype=np.float32).reshape(1, -1)
        D, I = index.search(query_emb, k)
        retrieved_texts = [texts[i] for i in I[0]]
        context = "\n".join(retrieved_texts)
        rag_prompt = f"Context:\n{context}\n\nQuestion: {prompt}\nAnswer:"
        strict_system_msg = (
            "You are a helpful assistant for customer experience analytics. "
            "Only answer based on the provided context you may summarize or synthesize from the context, but do not hallucinate."
        )
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": strict_system_msg},
                {"role": "user", "content": rag_prompt}
            ],
            max_tokens=256,
            temperature=0.2
        )
        answer = response.choices[0].message.content.strip()
        if answer.strip().lower().startswith("i don't know based on the search results"):
            print("\nNo answer found in search results. Here are the top retrieved review excerpts:\n")
            print(context)
        return answer
    else:
        print("Using structured RAG pipeline")
        # Use LLM to generate pandas code, then execute and return result
        code = generate_pandas_code(prompt)
        print(f"Generated pandas code: {code}")
        # Remove markdown code block formatting if present
        code = code.strip()
        # Remove triple backticks and language tags
        if code.startswith("```"):
            code = code.replace("```python", "").replace("```", "").strip()
        # Remove single backticks if present
        if code.startswith("`") and code.endswith("`"):
            code = code[1:-1].strip()
        # Remove any remaining lines starting with triple backticks
        code = "\n".join([line for line in code.splitlines() if not line.strip().startswith("```")]).strip()
        try:
            result = eval(code)
            # Handle empty results
            if hasattr(result, 'empty') and result.empty:
                result = "No results found for this query."
            elif isinstance(result, (pd.Series, pd.DataFrame)) and result.shape[0] == 0:
                result = "No results found for this query."
        except Exception as e:
            result = f"Error executing code: {e}"
        return result

print("Hybrid RAG pipeline ready: use hybrid_rag(prompt) for both semantic and structured queries.")

Hybrid RAG pipeline ready: use hybrid_rag(prompt) for both semantic and structured queries.


In [67]:
# Example: Operational CX query using RAG pipeline
sample_query = "List all departments mentioned in the reviews."
rag_answer = hybrid_rag(sample_query)
print(f"Q: {sample_query}\nA: {rag_answer}")

Intent detection using simple keywords: structured
Intent detection using pretrained classifier: structured
Using structured RAG pipeline
Generated pandas code: df['department'].unique()
Q: List all departments mentioned in the reviews.
A: ['Application' 'General' 'CustomerService' 'TransfersPayments'
 'FeesCharges' 'Cards' 'Offers' 'CASA' 'Fraud' 'Deposits' 'Investment']


In [68]:
# Example: Operational CX query using RAG pipeline
sample_query = "show department which had most complaints?"
rag_answer = hybrid_rag(sample_query)
print(f"Q: {sample_query}\nA: {rag_answer}")

Intent detection using simple keywords: structured
Intent detection using pretrained classifier: semantic
Using structured RAG pipeline
Generated pandas code: ```python
df[df['sentiment'] == 'Negative']['department'].value_counts().idxmax()
```
Q: show department which had most complaints?
A: Application


In [77]:
# Example: Operational CX query using RAG pipeline
sample_query = "List all apps with reviews in Arabic language."
rag_answer = hybrid_rag(sample_query)
print(f"Q: {sample_query}\nA: {rag_answer}")

Intent detection using simple keywords: structured
Intent detection using pretrained classifier: structured
Using structured RAG pipeline
Intent detection using pretrained classifier: structured
Using structured RAG pipeline
Generated pandas code: ```python
df[df['language'] != 'en']['app_name'].unique()
```
Q: List all apps with reviews in Arabic language.
A: []
Generated pandas code: ```python
df[df['language'] != 'en']['app_name'].unique()
```
Q: List all apps with reviews in Arabic language.
A: []


In [69]:
# Example: Operational CX query using RAG pipeline
sample_query = "which bank had most compliant SLA Status?"
rag_answer = hybrid_rag(sample_query)
print(f"Q: {sample_query}\nA: {rag_answer}")

Intent detection using simple keywords: structured
Intent detection using pretrained classifier: semantic
Using structured RAG pipeline
Generated pandas code: ```python
df[df['sla_status'] == 'SLA Met']['app_name'].value_counts().idxmax()
```
Q: which bank had most compliant SLA Status?
A: ADCB


In [70]:
# Example: Operational CX query using RAG pipeline
sample_query = "Count reviews with negative sentiment for each app."
rag_answer = hybrid_rag(sample_query)
print(f"Q: {sample_query}\nA: {rag_answer}")

Intent detection using simple keywords: structured
Intent detection using pretrained classifier: semantic
Using structured RAG pipeline
Generated pandas code: ```python
df[df['sentiment'] == 'Negative'].groupby('app_name').size()
```
Q: Count reviews with negative sentiment for each app.
A: app_name
ADCB        2234
ADIB         992
Citibank     827
DIB         2544
EI Bank      519
ENBD        5857
FAB         2451
HSBC         862
Mashreq     2870
RAKBANK     1179
SC Bank      295
dtype: int64


In [71]:
# Example: Semantic query using RAG pipeline (should trigger semantic search, not structured)
semantic_query = "What are some common themes or issues mentioned by users in their reviews?"
rag_semantic_answer = hybrid_rag(semantic_query)
print(f"Q: {semantic_query}\nA: {rag_semantic_answer}")

Intent detection using simple keywords: semantic
Intent detection using pretrained classifier: semantic
Using semantic RAG pipeline
Q: What are some common themes or issues mentioned by users in their reviews?
A: Some common themes or issues mentioned by users in their reviews include:

1. **General Issues**: Users frequently mention that the app has a lot of issues, indicating a broad dissatisfaction with its performance.

2. **New User Experience**: There are specific issues faced by new users, suggesting that the onboarding process or initial setup might be problematic.

3. **Download Problems**: Some users experience issues when downloading the app, which could point to technical difficulties or compatibility problems.

4. **Lack of Features and Functionality**: Users note a lack of features and limited functionality, indicating that the app may not meet their needs or expectations.

5. **Poor Performance**: Performance issues are highlighted, suggesting that the app may be slow or

In [72]:
# Example: Semantic query using RAG pipeline (should trigger semantic search, not structured)
semantic_query = "What are some common issues mentioned by users in their reviews related to cards?"
rag_semantic_answer = hybrid_rag(semantic_query)
print(f"Q: {semantic_query}\nA: {rag_semantic_answer}")

Intent detection using simple keywords: semantic
Intent detection using pretrained classifier: semantic
Using semantic RAG pipeline
Q: What are some common issues mentioned by users in their reviews related to cards?
A: Some common issues mentioned by users in their reviews related to cards include:

1. **Bugs and App Crashes**: Users report that the app frequently crashes, especially when trying to manage cards, and there are persistent bugs that have not been addressed for months.

2. **Lack of Information and Features**: Key card details such as statement date, payment date, last payment amount, and balance are difficult to locate. Users also mention the inability to send payments by IBAN, generate codes, or request support through the app.

3. **Missing Cards**: Certain types of cards, like student debit cards, do not appear on the home page, limiting access to functionalities like freezing the card or viewing transaction history.

4. **Hidden Charges and Miscommunication**: Users 

In [73]:
# RAG evaluation: test the hybrid_rag pipeline with example queries and expected answers

example_queries = [
    {
        "query": "List all departments mentioned in the reviews.",
        "expected_type": "structured"
    },
    {
        "query": "What do users say about mobile app performance?",
        "expected_type": "semantic"
    },
    {
        "query": "Count reviews with negative sentiment for each app.",
        "expected_type": "structured"
    },
    {
        "query": "Summarize feedback about customer support.",
        "expected_type": "semantic"
    }
]

for ex in example_queries:
    print(f"\nQuery: {ex['query']}")
    result = hybrid_rag(ex['query'])
    print(f"Result:\n{result}\n{'-'*60}")


Query: List all departments mentioned in the reviews.
Intent detection using simple keywords: structured
Intent detection using pretrained classifier: structured
Using structured RAG pipeline
Generated pandas code: `df['department'].unique()`
Result:
['Application' 'General' 'CustomerService' 'TransfersPayments'
 'FeesCharges' 'Cards' 'Offers' 'CASA' 'Fraud' 'Deposits' 'Investment']
------------------------------------------------------------

Query: What do users say about mobile app performance?
Intent detection using simple keywords: semantic
Intent detection using pretrained classifier: semantic
Using semantic RAG pipeline
Result:
Users have mixed feedback about the mobile app performance. Some users describe the app as having "excellent" performance and being the "best mobile app," while others report issues, mentioning that they are "experiencing slow mobile app" performance and that the app performance has "degraded significantly in the past one year or so."
-------------------

In [78]:
# RAG pipeline performance evaluation: expanded query set and metrics
import time

evaluation_queries = [
    {"query": "List all departments mentioned in the reviews.", "expected_type": "structured"},
    {"query": "What do users say about mobile app performance?", "expected_type": "semantic"},
    {"query": "Count reviews with negative sentiment for each app.", "expected_type": "structured"},
    {"query": "Summarize feedback about customer support.", "expected_type": "semantic"},
    {"query": "Which app has the highest average rating?", "expected_type": "structured"},
    {"query": "What are common complaints about card services?", "expected_type": "semantic"},
    {"query": "Show number of reviews by language.", "expected_type": "structured"},
    {"query": "Summarize feedback about login issues.", "expected_type": "semantic"},
    {"query": "List all unique SLA statuses in the reviews.", "expected_type": "structured"},
    {"query": "What do users say about app updates?", "expected_type": "semantic"},
    {"query": "Which department received the most positive reviews?", "expected_type": "structured"},
    {"query": "Summarize feedback about payment failures.", "expected_type": "semantic"},
    {"query": "Show average reply time by app.", "expected_type": "structured"},
    {"query": "What are common issues with onboarding?", "expected_type": "semantic"},
    {"query": "List all countries represented in the reviews.", "expected_type": "structured"},
    {"query": "Summarize feedback about security concerns.", "expected_type": "semantic"},
    {"query": "Show number of reviews by star rating.", "expected_type": "structured"},
    {"query": "What do users say about customer service response time?", "expected_type": "semantic"},
    {"query": "Which app store has more reviews?", "expected_type": "structured"},
    {"query": "Summarize feedback about app crashes.", "expected_type": "semantic"},
    {"query": "Show number of reviews per month for 2024.", "expected_type": "structured"},
    {"query": "What are common complaints about account access?", "expected_type": "semantic"},
    {"query": "List all apps with reviews in Arabic language.", "expected_type": "structured"},
    {"query": "Summarize feedback about notifications.", "expected_type": "semantic"},
    {"query": "Show number of reviews with SLA status 'Breached'.", "expected_type": "structured"},
    {"query": "Summarize feedback about app speed.", "expected_type": "semantic"},
    {"query": "Show reviews with star rating less than 3.", "expected_type": "structured"},
    {"query": "What do users say about password reset process?", "expected_type": "semantic"},
    {"query": "List all apps reviewed in 2025.", "expected_type": "structured"},
    {"query": "Summarize feedback about transaction failures.", "expected_type": "semantic"},
    {"query": "Show number of reviews by department.", "expected_type": "structured"},
    {"query": "What are common complaints about mobile deposits?", "expected_type": "semantic"},
    {"query": "List all reviews from India.", "expected_type": "structured"},
    {"query": "Summarize feedback about app navigation.", "expected_type": "semantic"},
    {"query": "Show reviews with reply time greater than 24 hours.", "expected_type": "structured"},
    {"query": "What do users say about push notifications?", "expected_type": "semantic"},
    {"query": "List all reviews with sentiment 'Neutral'.", "expected_type": "structured"},
    {"query": "Summarize feedback about multi-factor authentication.", "expected_type": "semantic"},
    {"query": "Show reviews for app 'BankX'.", "expected_type": "structured"},
    {"query": "What are common complaints about customer onboarding?", "expected_type": "semantic"},
    {"query": "List all reviews with SLA status 'Met'.", "expected_type": "structured"},
    {"query": "Summarize feedback about mobile check deposit.", "expected_type": "semantic"},
    {"query": "Show reviews by month for app 'BankY'.", "expected_type": "structured"},
    {"query": "What do users say about app reliability?", "expected_type": "semantic"},
    {"query": "List all reviews with language 'English'.", "expected_type": "structured"},
    {"query": "Summarize feedback about customer loyalty programs.", "expected_type": "semantic"},
    {"query": "Show reviews with star rating 5.", "expected_type": "structured"},
    {"query": "What are common complaints about mobile payments?", "expected_type": "semantic"},
    {"query": "List all reviews with reply ETA less than 2 hours.", "expected_type": "structured"},
    {"query": "Summarize feedback about app design.", "expected_type": "semantic"}
]

results = []
for ex in evaluation_queries:
    start_time = time.time()
    intent = resolve_intent(ex['query'])
    result = hybrid_rag(ex['query'])
    latency = time.time() - start_time
    results.append({
        'query': ex['query'],
        'expected_type': ex['expected_type'],
        'detected_intent': intent,
        'result': result,
        'latency_sec': latency
    })

# Calculate metrics
total = len(results)
correct_intent = sum(r['expected_type'] == r['detected_intent'] for r in results)
accuracy = correct_intent / total
avg_latency = sum(r['latency_sec'] for r in results) / total
error_count = sum(isinstance(r['result'], str) and r['result'].startswith('Error') for r in results)
error_rate = error_count / total

print(f"RAG Pipeline Evaluation Results:")
for r in results:
    print(f"Query: {r['query']}")
    print(f"Expected: {r['expected_type']}, Detected: {r['detected_intent']}, Latency: {r['latency_sec']:.2f}s")
    print(f"Result: {r['result']}")
    print('-'*60)

print(f"\nIntent Detection Accuracy: {accuracy:.2%}")
print(f"Average Latency: {avg_latency:.2f} seconds")
print(f"Error Rate: {error_rate:.2%}")

# Coverage: percent of queries answered (not error)
answered_count = sum(not (isinstance(r['result'], str) and r['result'].startswith('Error')) for r in results)
coverage = answered_count / total
print(f"Coverage: {coverage:.2%}")

Intent detection using simple keywords: structured
Intent detection using pretrained classifier: structured
Intent detection using simple keywords: structured
Intent detection using pretrained classifier: structured
Using structured RAG pipeline
Intent detection using pretrained classifier: structured
Using structured RAG pipeline
Generated pandas code: `df['department'].unique()`
Intent detection using simple keywords: semantic
Intent detection using pretrained classifier: semantic
Intent detection using simple keywords: semantic
Intent detection using pretrained classifier: semantic
Using semantic RAG pipeline
Generated pandas code: `df['department'].unique()`
Intent detection using simple keywords: semantic
Intent detection using pretrained classifier: semantic
Intent detection using simple keywords: semantic
Intent detection using pretrained classifier: semantic
Using semantic RAG pipeline
Intent detection using simple keywords: structured
Intent detection using pretrained classifi

In [ ]:
## Summary

# Intent Detection Accuracy (98.00%): Out of all queries, 98% were correctly classified as either "structured" or "semantic" by intent detection logic. This shows pipeline is highly effective at routing queries to the right processing path.

# Average Latency (2.16 seconds): On average, each query took 2.16 seconds to process and return a result. This includes intent detection, retrieval, LLM code generation, and execution.

# Error Rate (6.00%): 6% of queries resulted in an error (e.g., code execution failed, invalid output, or other exceptions). Lower error rates indicate more robust code generation and execution. There is scope for improvement.

# Coverage (94.00%): 94% of queries received a valid answer (not an error). High coverage means pipeline can successfully handle most user queries. However there is scope for improvement.